In [7]:
#pip install ortools pandas openpyxl

from ortools.sat.python import cp_model
import pandas as pd
from datetime import datetime, timedelta

start_date = datetime.strptime('2023-07-01', '%Y-%m-%d')

# Create the model.
model = cp_model.CpModel()

## Constants
num_employees = 6
num_days = 365
shift_types = ['Morning', 'Afternoon', 'Night', 'Day off']

# Variables
shifts = {}
total_shifts = []
for j in range(num_employees):
    for i in range(num_days):
        for k in range(len(shift_types)):
            shift_var = model.NewBoolVar('shift_%i%i%i' % (j, i, k))
            shifts[(j, i, k)] = shift_var
            if k != 3:  # Don't count 'Day off' as a shift
                total_shifts.append(shift_var)

# Constraints

# Each shift every day is assigned to exactly one employee,
# or the employee has a day off
for i in range(num_days):
    for k in range(shifts_per_day):
        model.Add(sum(shifts[(j, i, k)] for j in range(num_employees)) == 1)

# No employee works two shifts in the same day
for j in range(num_employees):
    for i in range(num_days):
        model.Add(sum(shifts[(j, i, k)] for k in range(len(shift_types))) <= 1)

# Each employee works no more than 5 days in a row
for j in range(num_employees):
    for i in range(num_days - 5):
        model.Add(sum(shifts[(j, i + d, k)] for d in range(6) for k in range(shifts_per_day)) <= 5)

# Each employee works no more than 2 nights in a row
for j in range(num_employees):
    for i in range(num_days - 3):
        model.Add(sum(shifts[(j, i + d, 2)] for d in range(3)) <= 2)

# Minimize the total number of shifts
model.Minimize(sum(total_shifts))

# Creates the solver and solve.
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print the results and Prepare the data
roster_data = []
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('Found solution!')
    for i in range(num_days):
        for j in range(num_employees):
            worked = False
            for k in range(len(shift_types)):
                if solver.BooleanValue(shifts[(j, i, k)]):
                    print(f"Employee {j} works on day {i} shift {shift_types[k]}")
                    current_date = (start_date + timedelta(days=i)).strftime('%Y-%m-%d')
                    roster_data.append([current_date, shift_types[k], j])
                    worked = True
            if not worked: # If no shift was worked, add a day off
                roster_data.append([current_date, 'Day off', j])
else:
    print('No solution found !')

# Create a pandas DataFrame
roster_df = pd.DataFrame(roster_data, columns=['Date', 'Shift', 'Employee'])

# Pivot DataFrame so that shifts are columns
roster_pivot_df = roster_df.pivot_table(index='Date', columns='Shift', values='Employee', aggfunc='first', fill_value='')

# Ensure 'Day off' is the last column
cols = roster_pivot_df.columns.tolist()
cols.remove('Day off')
cols.append('Day off')
roster_pivot_df = roster_pivot_df[cols]

# Write to Excel
roster_pivot_df.to_excel('roster.xlsx')



Found solution!
Employee 0 works on day 0 shift Night
Employee 4 works on day 0 shift Morning
Employee 5 works on day 0 shift Afternoon
Employee 0 works on day 1 shift Night
Employee 1 works on day 1 shift Afternoon
Employee 5 works on day 1 shift Morning
Employee 2 works on day 2 shift Night
Employee 4 works on day 2 shift Morning
Employee 5 works on day 2 shift Afternoon
Employee 2 works on day 3 shift Afternoon
Employee 3 works on day 3 shift Morning
Employee 4 works on day 3 shift Night
Employee 0 works on day 4 shift Morning
Employee 1 works on day 4 shift Afternoon
Employee 2 works on day 4 shift Night
Employee 1 works on day 5 shift Morning
Employee 2 works on day 5 shift Afternoon
Employee 4 works on day 5 shift Night
Employee 2 works on day 6 shift Night
Employee 3 works on day 6 shift Afternoon
Employee 5 works on day 6 shift Morning
Employee 3 works on day 7 shift Night
Employee 4 works on day 7 shift Afternoon
Employee 5 works on day 7 shift Morning
Employee 0 works on day 